In [20]:
!pip install -q chromadb
!pip install -q torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu121
!pip install -q transformers
!pip install -q langchain
!pip install -q sentence_transformers
!pip install -q pypdf
!pip install -q unstructured
!pip install -U -q langchain-community

In [2]:
!transformers-cli env


Copy-and-paste the text below in your GitHub issue and FILL OUT the two last points.

- `transformers` version: 4.41.1
- Platform: Windows-10-10.0.22621-SP0
- Python version: 3.11.9
- Huggingface_hub version: 0.23.1
- Safetensors version: 0.4.3
- Accelerate version: not installed
- Accelerate config: not found
- PyTorch version (GPU?): 2.3.0+cu121 (True)
- Tensorflow version (GPU?): not installed (NA)
- Flax version (CPU?/GPU?/TPU?): not installed (NA)
- Jax version: not installed
- JaxLib version: not installed
- Using GPU in script?: <fill in>
- Using distributed or parallel set-up in script?: <fill in>



In [1]:
import os
import torch
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    pipeline,
    AutoModel,
)

C:\Users\201902452\AppData\Local\Programs\Python\Python311\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [85]:
#################################################################
# Tokenizer
#################################################################

model_name='meta-llama/Meta-Llama-3-8B-Instruct'
# Path to folder containing the model
#model_name = "C:\Users\201902452\Downloads\Meta-Llama-3-8B-Instruct\Meta-Llama-3-8B-Instruct" 

tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

OSError: We couldn't connect to 'https://huggingface.co' to load this file, couldn't find it in the cached files and it looks like meta-llama/Meta-Llama-3-8B-Instruct is not the path to a directory containing a file named config.json.
Checkout your internet connection or see how to run the library in offline mode at 'https://huggingface.co/docs/transformers/installation#offline-mode'.

In [21]:
#################################################################
# Load pre-trained config
#################################################################
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    low_cpu_mem_usage = True,
    device_map = device_map
)

NameError: name 'bnb_config' is not defined

In [2]:
tokenizer = AutoTokenizer.from_pretrained("C:/Users/201902452/Downloads/Meta-Llama-3-8B-Instruct/Meta-Llama-3-8B-Instruct")

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [3]:
model = AutoModel.from_pretrained("C:/Users/201902452/Downloads/Meta-Llama-3-8B-Instruct/Meta-Llama-3-8B-Instruct")

Loading checkpoint shards: 100%|█████████████████████████████████████████████████████████| 4/4 [00:07<00:00,  1.85s/it]


In [90]:
from transformers import AutoModel
from safetensors import safe_open

file_paths = [
    "C:/Users/201902452/Downloads/Meta-Llama-3-8B-Instruct/Meta-Llama-3-8B-Instruct/model-00001-of-00004.safetensors",
    "C:/Users/201902452/Downloads/Meta-Llama-3-8B-Instruct/Meta-Llama-3-8B-Instruct/model-00002-of-00004.safetensors",
    "C:/Users/201902452/Downloads/Meta-Llama-3-8B-Instruct/Meta-Llama-3-8B-Instruct/model-00003-of-00004.safetensors",
    "C:/Users/201902452/Downloads/Meta-Llama-3-8B-Instruct/Meta-Llama-3-8B-Instruct/model-00004-of-00004.safetensors"
]

for file_path in file_paths:
    try:
        with safe_open(file_path, framework="pt") as f:
            print(f"File {file_path} loaded successfully.")
    except Exception as e:
        print(f"Error loading file {file_path}: {e}")


File C:/Users/201902452/Downloads/Meta-Llama-3-8B-Instruct/Meta-Llama-3-8B-Instruct/model-00001-of-00004.safetensors loaded successfully.
File C:/Users/201902452/Downloads/Meta-Llama-3-8B-Instruct/Meta-Llama-3-8B-Instruct/model-00002-of-00004.safetensors loaded successfully.
File C:/Users/201902452/Downloads/Meta-Llama-3-8B-Instruct/Meta-Llama-3-8B-Instruct/model-00003-of-00004.safetensors loaded successfully.
File C:/Users/201902452/Downloads/Meta-Llama-3-8B-Instruct/Meta-Llama-3-8B-Instruct/model-00004-of-00004.safetensors loaded successfully.


## LangChain

In [4]:
# Paths to the database and the data

CHROMA_PATH = "chroma"
DATA_PATH = "data"

In [5]:
from bs4 import BeautifulSoup
import os

def XMLLoader(folder_path):
    reportData_list = []

    for filename in os.listdir(folder_path):
        if filename.endswith('.xml'):
            file_path = os.path.join(folder_path, filename)
            with open(file_path, 'r') as file:
                content = file.read()

            soup = BeautifulSoup(content, 'xml')

            reportData = {}
            for tag in soup.find_all():
                if tag.name != "report":
                    reportData[tag.name] = tag.text.strip()

            reportData["filename"] = filename
            reportData_list.append(reportData)

    return reportData_list

In [6]:
data =  XMLLoader("data")

In [14]:
from langchain.docstore.document import Document

documents = []
for doc in data:
    
    document = Document(page_content=doc["report_text"], metadata={"source": doc["filename"], 
                                                                   "subtype": doc["subtype"], 
                                                                   "type": doc["type"], 
                                                                   "chief_complaint": doc["chief_complaint"], 
                                                                   "admit_diagnosis": doc["admit_diagnosis"], 
                                                                   "discharge_diagnosis": doc["discharge_diagnosis"], 
                                                                   "year": doc["year"], 
                                                                   "downlaod_time": doc.get("download_time", ""), # Algunos no tiemem este campo
                                                                   "deid": doc["deid"]})
    documents.append(document)

In [42]:
#documents[1].metadata["subtype"]="TEST"
#print(documents[7])

¿Porque dividir en chunks?

Aunque lo ideal sería meterle el documento entero al LLM para que tenga el contexto completo, muchos LLM tienen una ventana de contexto limitada. Además existe el problema "lost-in-the-middle" (el LLM prestará mas atención al inicio del documento y al final) o "needle-on-a-hysack", en el que el LLM pierde información que se encuentra entre medias.

Si los chunks son muy pequeños, existe la posibilidad de que el modelo no tenga el contexto completo.

Trade-off:
Chunks pequeños: Mas granularidad, sources mas diversas, contexto incompleto
Chunks grandes: Menos granularidad, sources menos diversas, lost in the midle problem

Depende de:
- Embedding model
- Context length
- Type of documents

In [15]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

# LLama3 - Context window of 8000 tokens
def split_text(documents):
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=512, #High impact on performance
        chunk_overlap=200,
        length_function=len,
        add_start_index=True
    )
    
    chunks = text_splitter.split_documents(documents)
    print(f"Split {len(documents)} documents into {len(chunks)} chunks.")
    
    return chunks

In [16]:
chunks = split_text(documents)

# Cogemos un chunk aleatorio
document = chunks[10]
print(document.page_content) # Printeamos su contenido
print(document.metadata) # Y su metadata (Fichero al que pertenece y donde empieza 'start_index')

Split 39 documents into 545 chunks.
3. D5 half normal saline with 20 of K at 80 cc an hour.
4. Folic acid 1 mg daily.
5. Lasix 80 mg daily.
6. Heparin 5000 units subq q.12 h. 
7. Lantus insulin.
8. Sliding scale regular insulin.
9. Synthroid 50 mcg daily.
10. Metoprolol 25 mg daily.
11. Flagyl 500 mg IV q.8 h. 
12. Avelox 400 mg IV q. 24 h. 
13. Pravachol 20 mg daily.
14. Diovan 80 mg daily.
 
LABORATORY DATA 
Blood cultures are no growth today.  Sodium 137, potassium 4.4, chloride 109,
{'source': 'report10.xml', 'subtype': 'CONSULT', 'type': 'HP', 'chief_complaint': 'DIVERTICULITIS', 'admit_diagnosis': '562.10', 'discharge_diagnosis': '584.9,518.0,403.91,428.0,486,276.2,562.11,585.4,518.3,244.9,414.01,250.80,791.0,250.50,362.01,285.29,', 'year': '2007', 'downlaod_time': '', 'deid': 'v.6.22.07.0', 'start_index': 2901}


In [21]:
from langchain.vectorstores.chroma import Chroma
from langchain.embeddings import SentenceTransformerEmbeddings
import shutil

def save_to_chroma(chunks):
    
    # Clear out the database first if already exists.
    if os.path.exists(CHROMA_PATH):
        shutil.rmtree(CHROMA_PATH)

    # https://huggingface.co/sentence-transformers
    # https://huggingface.co/spaces/mteb/leaderboard
    #embeddings = HuggingFaceEmbeddings(model_name='sentence-transformers/all-mpnet-base-v2')
    embeddings = SentenceTransformerEmbeddings(model_name="all-MiniLM-L6-v2")

    # Estos dos parámetros determinarán el tamaño de los chunks
    # Embedding dimensions: 384
    # Max tokens: 512 (Este determinará el tamaño de los chunks)
    
    print(f"Model's max seq length: {SentenceTransformerEmbeddings('all-MiniLM-L6-v2').max_seq_length}")  # Importante
    
    # Create a new DB from the documents.
    db = Chroma.from_documents(
        chunks, 
        embeddings, 
        persist_directory=CHROMA_PATH
    )
    
    db.persist()
    print(f"Saved {len(chunks)} chunks to {CHROMA_PATH}.")

    return embeddings

In [22]:
embeddings = save_to_chroma(chunks)

No sentence-transformers model found with name sentence-transformers/all-MiniLM-L6-v2. Creating a new one with MEAN pooling.
C:\Users\201902452\AppData\Local\Programs\Python\Python311\Lib\site-packages\huggingface_hub\file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


OSError: We couldn't connect to 'https://huggingface.co' to load this file, couldn't find it in the cached files and it looks like sentence-transformers/all-MiniLM-L6-v2 is not the path to a directory containing a file named config.json.
Checkout your internet connection or see how to run the library in offline mode at 'https://huggingface.co/docs/transformers/installation#offline-mode'.

### Evaluator (For testing purposes)

In [134]:
#Function to turn the word apple to a vector
from langchain.evaluation import load_evaluator

vector = embeddings.embed_query("almendras")
#print(f"Vector for 'apple': {vector}") # Algunos pueden ser muy largos
print(f"Vector length: {len(vector)}")

# Compare vector of two words (distance between two words)
# Esto nos ayudará a saber si dos palabras estas correlacionadas
# Si comparamos dos palabras iguales, la distancia será prácticamente 0
# Utilza por defecto OpenAI embeddings!
evaluator = load_evaluator("embedding_distance", embeddings=embeddings)

words = ("apple", "iphone")
x = evaluator.evaluate_strings(prediction=words[0], reference=words[1])
print(f"Comparing ({words[0]}, {words[1]}): {x}")

Vector length: 512
Comparing (apple, iphone): {'score': 0.30508241313171336}


In [50]:
from langchain.llms import HuggingFacePipeline

# Pipeline for inference
def load_model():
    text_generation_pipeline = pipeline(
        model=model, # Model loaded in the first part
        tokenizer=tokenizer, # Tokenizer loaded in the first part
        task="text-generation",
        temperature=0.1,
        repetition_penalty=1.1,
        return_full_text=True,
        max_new_tokens=300,
        do_sample=True
    )

    hf = HuggingFacePipeline(pipeline=text_generation_pipeline)
    return hf

In [136]:
llm = load_model()

In [137]:
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
from langchain_core.runnables import RunnablePassthrough, RunnableParallel
from langchain_core.output_parsers import StrOutputParser

def query_data(embeddings, llm):
    
    db = Chroma(persist_directory=CHROMA_PATH, embedding_function=embeddings)
    
    retriever = db.as_retriever(
        search_kwargs = {"k":3} # Para obtener los top 3 documentos mas reelevantes (Reranking)
    )

    prompt_template = """
    ### [INST]
    Act as an expert doctor. Use the medical reports provided below to answer the question.

    If you do not know the answer, or are unsure, say you don't know.
    
    {context}
    
    ### QUESTION:
    {question}
    
    [/INST]
     """
    
    promptTemplate = PromptTemplate(
        input_variables=["context", "question"],
        template=prompt_template
    )
    
    #llm_chain = LLMChain(llm=llm, prompt=promptTemplate)

    # RunnablePassthrough para pasar la query al siguiente step en la chain
    #rag_chain = ( 
    # {"context": retriever, "question": RunnablePassthrough()}
    #    | llm_chain
    #)



    setup_and_retrieval = RunnableParallel({"question": RunnablePassthrough(), "context": retriever })
    output_parser = StrOutputParser()

    # LCEL language to create the chain (de izquierda a derecha el output de uno se pasa al otro)
    parent_retrieval_chain = setup_and_retrieval | promptTemplate | llm | output_parser

    return parent_retrieval_chain

In [ ]:
chain = query_data(embeddings, llm)

In [ ]:

out = chain.invoke("Wich patient has a history of cholecystectomy")
print(out)

In [149]:
#rag_chain = query_data(embeddings, llm)
#query = """Explícame el artículo 32. Acceso a números o servicios"""
#answer = rag_chain.invoke(query)

In [150]:
#answer['context']

[Document(page_content='mecanismos  de notificación  a que se refieren  los artículos  32, 33 y 34 del Código \nBOLETÍN OFICIAL DEL ESTADO\nNúm. 155 Miércoles 29 de junio de 2022 Sec. I.   Pág. 91386\ncve: BOE-A-2022-10757\nVerificable en https://www.boe.es', metadata={'page': 133, 'source': 'data/BOE-A-2022-10757.pdf', 'start_index': 0}),
 Document(page_content='que corresponda.\nBOLETÍN OFICIAL DEL ESTADO\nNúm. 155 Miércoles 29 de junio de 2022 Sec. I.   Pág. 91287\ncve: BOE-A-2022-10757\nVerificable en https://www.boe.es', metadata={'page': 34, 'source': 'data/BOE-A-2022-10757.pdf', 'start_index': 0}),
 Document(page_content='cualquier persona física o jurídica que desee contribuir, desinteresadamente, a la \nfinanciación de cualquier prestación propia del servicio universal.\nLos operadores sujetos a obligaciones de prestación del servicio universal recibirán \nde este fondo la cantidad correspondiente al coste neto que les supone dicha obligación, \ncalculado según el procedimient

In [151]:
#answer['text']

"\n    ### [INST] Instrucción: Eres un experto en jurisdiccion, responde a la pregunta según tus conocimientos de jurisdiccion y el siguiente contexto:\n    \n    [Document(page_content='mecanismos  de notificación  a que se refieren  los artículos  32, 33 y 34 del Código \\nBOLETÍN OFICIAL DEL ESTADO\\nNúm. 155 Miércoles 29 de junio de 2022 Sec. I.   Pág. 91386\\ncve: BOE-A-2022-10757\\nVerificable en https://www.boe.es', metadata={'page': 133, 'source': 'data/BOE-A-2022-10757.pdf', 'start_index': 0}), Document(page_content='que corresponda.\\nBOLETÍN OFICIAL DEL ESTADO\\nNúm. 155 Miércoles 29 de junio de 2022 Sec. I.   Pág. 91287\\ncve: BOE-A-2022-10757\\nVerificable en https://www.boe.es', metadata={'page': 34, 'source': 'data/BOE-A-2022-10757.pdf', 'start_index': 0}), Document(page_content='cualquier persona física o jurídica que desee contribuir, desinteresadamente, a la \\nfinanciación de cualquier prestación propia del servicio universal.\\nLos operadores sujetos a obligaciones 

In [158]:
indent = 0
for key, value in answer.items():
    if isinstance(value, dict):
        print('  ' * indent + f'{key}:')
        pretty_print_dict(value, indent + 1)
    else:
        print('  ' * indent + f'{key}: {value}')

context: [Document(page_content='mecanismos  de notificación  a que se refieren  los artículos  32, 33 y 34 del Código \nBOLETÍN OFICIAL DEL ESTADO\nNúm. 155 Miércoles 29 de junio de 2022 Sec. I.   Pág. 91386\ncve: BOE-A-2022-10757\nVerificable en https://www.boe.es', metadata={'page': 133, 'source': 'data/BOE-A-2022-10757.pdf', 'start_index': 0}), Document(page_content='que corresponda.\nBOLETÍN OFICIAL DEL ESTADO\nNúm. 155 Miércoles 29 de junio de 2022 Sec. I.   Pág. 91287\ncve: BOE-A-2022-10757\nVerificable en https://www.boe.es', metadata={'page': 34, 'source': 'data/BOE-A-2022-10757.pdf', 'start_index': 0}), Document(page_content='cualquier persona física o jurídica que desee contribuir, desinteresadamente, a la \nfinanciación de cualquier prestación propia del servicio universal.\nLos operadores sujetos a obligaciones de prestación del servicio universal recibirán \nde este fondo la cantidad correspondiente al coste neto que les supone dicha obligación, \ncalculado según el proce